In [16]:
from numpy import array
from keras.models import Sequential
from keras.models import Model
from keras.layers import LSTM, Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, RepeatVector, TimeDistributed, Reshape
from keras.utils import plot_model
from keras.optimizers import RMSprop
import ffmpeg
import numpy as np
from generator import MyGenerator
from data import Data

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def get_autoencoder(inputs):                             

        
    #encoder
    conv1 = TimeDistributed(Conv2D(16, (8, 8), strides=(4,4), activation='relu', padding='same'))(inputs)
    pool1 = TimeDistributed(MaxPooling2D(pool_size=(4, 4)))(conv1) #200 x 200 x 32
    
    print(pool1.shape)
    
    reshape = Reshape((inputs.shape[1], 10000))(pool1)
    dense1 = TimeDistributed(Dense(100))(reshape)
    lstm1 = LSTM(100, return_sequences=True)(dense1)
    
    lstm2 = LSTM(100, return_sequences=True)(lstm1)
    dense2 = TimeDistributed(Dense(10000))(lstm2)
    reshape2 = Reshape((-1, 100, 100, 1))(dense2)
     
        
    conv5 = TimeDistributed(Conv2D(16, (8, 8), strides=(1,1), activation='relu', padding='same'))(reshape2) # 200 x 200 x 16
    up2 = TimeDistributed(UpSampling2D((4,4)))(conv5) # 400 x 400 x 16
    decoded = TimeDistributed(Conv2D(1, (3, 3), strides=(1,1), activation='relu', padding='same'))(up2)
    
    ### BIIIG NETWORK HERE!!111!
    
    #conv1 = TimeDistributed(Conv2D(32, (3, 3), strides=(1,1), activation='relu', padding='same'))(inputs)
    #pool1 = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(conv1) #200 x 200 x 32
    #conv2 = TimeDistributed(Conv2D(16, (3, 3), strides=(1,1), activation='relu', padding='same'))(pool1) #200 x 200 x 16
    #pool2 = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(conv2) #100 x 100 x 16
    #conv3 = TimeDistributed(Conv2D(8, (3, 3), strides=(1,1), activation='relu', padding='same'))(pool2) #100 x 100 x 8    
    
    #reshape = Reshape((inputs.shape[1], 80000))(conv3)
    #dense1 = TimeDistributed(Dense(100))(reshape)
    #lstm1 = LSTM(100, return_sequences=True)(dense1)
   
    #decoder
    #lstm2 = LSTM(100, return_sequences=True)(lstm1)
    #dense2 = TimeDistributed(Dense(80000))(lstm2)
    #reshape2 = Reshape((-1, 100, 100, 8))(dense2)
    #conv4 = TimeDistributed(Conv2D(8, (3, 3), strides=(1,1), activation='relu', padding='same'))(reshape2) #100 x 100 x 8
    #up1 = TimeDistributed(UpSampling2D((2,2)))(conv4) # 200 x 200 x 128
    #conv5 = TimeDistributed(Conv2D(16, (3, 3), strides=(1,1), activation='relu', padding='same'))(up1) # 200 x 200 x 16
    #up2 = TimeDistributed(UpSampling2D((2,2)))(conv5) # 400 x 400 x 16
    #decoded = TimeDistributed(Conv2D(1, (3, 3), strides=(1,1), activation='relu', padding='same'))(up2) # 400 x 400 x 1
    return decoded


In [17]:
import tensorflow as tf
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

input_dims = (20, 400, 400, 1)
inputs = Input(shape=input_dims)
inputs2 = Input(shape=input_dims)
batch_size = 32
shuffle = True

data = Data.get_data_paths("./calving")
data_generator = MyGenerator(data, batch_size=batch_size, dim=input_dims, shuffle=shuffle)
print(inputs.shape[2])

autoencoder = Model(inputs, get_autoencoder(inputs))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop(), options=run_opts) 
autoencoder.summary()





400
(?, 20, 25, 25, 16)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 20, 400, 400, 1)   0         
_________________________________________________________________
time_distributed_26 (TimeDis (None, 20, 100, 100, 16)  1040      
_________________________________________________________________
time_distributed_27 (TimeDis (None, 20, 25, 25, 16)    0         
_________________________________________________________________
reshape_9 (Reshape)          (None, Dimension(20), 100 0         
_________________________________________________________________
time_distributed_28 (TimeDis (None, Dimension(20), 100 1000100   
_________________________________________________________________
lstm_7 (LSTM)                (None, Dimension(20), 100 80400     
_________________________________________________________________
lstm_8 (LSTM)                (None, Dimension(20), 1

In [ ]:
autoencoder.fit_generator(generator=data_generator, max_queue_size=1)

Epoch 1/1
